In [2]:
select = 0

import SimpleITK as sitk

#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, fixed
import os
import gui
import resample_itk_image
from myshow import myshow, myshow3d
from itertools import chain
import pandas as pd

%matplotlib widget

# PC
data_folder = r"C:\Users\david\UW\Ryan Kellogg - Kempe files\subjectData"
data_folder = os.path.abspath(os.path.expanduser(os.path.expandvars(data_folder)))

# create output directories if they don't exist

#for root, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
#    if root[len(data_folder):].count(os.sep) == 2:
#        if not (os.path.exists(os.path.join(root,'output'))):
#            os.mkdir(os.path.join(root,'output'))

# generate structure of files 

data_subj_list = list()
output_dir_list = list()
for data_subj, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
    if (data_subj[len(data_folder):].count(os.sep) == 3) and not 'output' in data_subj[len(data_folder):]:
       # print(data_subj)
        super_folder = data_subj.rsplit('\\',1)
        OUTPUT_DIR = os.path.join(super_folder[0],'output')
        
        data_subj_list.append(data_subj)
        output_dir_list.append(OUTPUT_DIR)

length_data = len(data_subj_list)

# Read the original series. First obtain the series file names using the
# image series reader.
data_subj = data_subj_list[select]
OUTPUT_DIR = output_dir_list[select]
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_subj)
if not series_IDs:
    print("ERROR: given directory \""+data_folder+"\" does not contain a DICOM series.")

series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_subj, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time).
# By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the
# private ones.
series_reader.MetaDataDictionaryArrayUpdateOn()
#series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName(series_file_names[0])
#reader.LoadPrivateTagsOn();

reader.ReadImageInformation();

#for k in reader.GetMetaDataKeys():
#    v = reader.GetMetaData(k)
#    print("({0}) = = \"{1}\"".format(k,v))

print('Original Image Spacing ',image3D.GetSpacing())
print('Original Image Size ', image3D.GetSize())

orig_size = image3D.GetSize()
orig_spacing = image3D.GetSpacing()
min_orig = min(orig_spacing)
max_orig = max(orig_spacing)

intermed_spacing = 2.0
image3D_resample = resample_itk_image.resample_img(image3D,[intermed_spacing,intermed_spacing,intermed_spacing])

print('Resampled Image Spacing ',image3D_resample.GetSpacing())
print('Resampled Image Size ', image3D_resample.GetSize())

image3D_resample_seq = resample_itk_image.resample_img(image3D,[min_orig,min_orig,min_orig])

print('Resampled Image Spacing For Sequential ',image3D_resample_seq.GetSpacing())
print('Resampled Image Size For Sequential', image3D_resample_seq.GetSize())

# recast types as necessary for analysis 

recast_image3D = sitk.Cast(sitk.RescaleIntensity(image3D_resample), sitk.sitkUInt8)
recast32_image3D = sitk.Cast(image3D_resample, sitk.sitkFloat32)
recast32_image3D_seq = sitk.Cast(image3D_resample_seq, sitk.sitkFloat32)

initial_seed_point_indexes = np.loadtxt(os.path.join(OUTPUT_DIR,'seeds.csv'),delimiter=',')
initial_seed_point_indexes = initial_seed_point_indexes.astype(int)
initial_seed_point_indexes = initial_seed_point_indexes.tolist()

seg = sitk.Image(image3D_resample.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(image3D_resample)
for index in range(1,len(initial_seed_point_indexes)):
    seg[initial_seed_point_indexes[index]] = 1
seg = sitk.BinaryDilate(seg, 10)

myshow(sitk.LabelOverlay(recast_image3D, seg))

Original Image Spacing  (0.369140625, 0.36927621861152, 2.000081777572632)
Original Image Size  (512, 677, 108)
Resampled Image Spacing  (2.0, 2.0, 2.0)
Resampled Image Size  (94, 125, 108)
Resampled Image Spacing For Sequential  (0.369140625, 0.369140625, 0.369140625)
Resampled Image Size For Sequential (512, 677, 585)


interactive(children=(IntSlider(value=53, description='z', max=107), Output()), _dom_classes=('widget-interact…